Feed Forward Neural Network

Tested using the MNIST dataset

Accuracy of 95% using 1 hidden layer

In [10]:
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import time

In [11]:
#Features not yet included: Relu, Cost Plotting.
class FFNN():
    
    def __init__(self, hidden_layers, activation_functions, cost_function, data, target):
        """
        "hidden_layers" should be a list of hidden layer sizes in ascending order
        "activation_functions" should be a list of activation functions for layer hidden_1 through output
        "cost_function" should be either "Neg Log" or "Sum of Squares"
        "data" should be a numpy array with m data point rows and n feature columns
        "target" should be a numpy array with m data point rows and a column per class
        """
        self.sizes = [np.shape(data)[1]] + hidden_layers + [np.shape(target)[1]] #List of layer sizes
        self.activation_functions = activation_functions
        self.cost_function = cost_function
        self.num_classes = target.shape[1]
        self.data = np.hstack((data, target))
        
        self.weights = [np.random.normal(scale = 1/np.sqrt(prev), size = (nex, prev)) \
                        for prev, nex in zip(self.sizes[:-1], self.sizes[1:])]
        self.biases = [np.zeros((i,1)) for i in self.sizes[1:]]
    
    # Split train_data, train_target, test_data, test_target
    def create_val_set(self, val_frac):
        np.random.shuffle(self.data)
        split = int(len(self.data)*(val_frac))
        self.val_data = self.data[0:split, :]
        self.data = self.data[split:, :]
        
    # Calculate the standardization metrics from the training data
    def initial_standardization(self):
        self.data_means = np.mean(self.data[:,:-self.num_classes], axis=0)
        self.data_scale = np.std(self.data[:,:-self.num_classes], axis=0)
        self.data[:,:-self.num_classes] = self.standardizeFunction(self.data[:,:-self.num_classes])
        if self.val_frac:
            self.val_data[:,:-self.num_classes] = self.standardizeFunction(self.val_data[:,:-self.num_classes])
        
    def standardizeFunction(self, data):
        out = data
        for column in range(data.shape[1]):
            if self.data_scale[column]:
                out[:,column] = (data[:,column] - self.data_means[column]) / self.data_scale[column]
        return out
    
    def train(self, batch_size, alpha, epochs, val_frac = 0, standardize = True, regularization = 0):
        self.batch_size = batch_size
        self.val_frac = val_frac
        self.standardize = standardize
        self.reg = regularization
        if self.val_frac:
            self.create_val_set(val_frac)
            self.val_acc = np.zeros(epochs)
        if self.standardize:
            self.initial_standardization()
        self.activations = [np.zeros((layer_size, batch_size)) for layer_size in self.sizes]
        for epoch in range(epochs):
            print('epoch ', epoch+1)
            np.random.shuffle(self.data) #Shuffle the data before each epoch
            for minibatch in range(int(len(self.data)/self.batch_size)):
                minibatch_data = self.data[minibatch*self.batch_size : (minibatch+1)*self.batch_size, :]
                inputs = minibatch_data[:,:-self.num_classes].transpose()
                target = minibatch_data[:,-self.num_classes:].transpose()
                self.feedforward(inputs)
                self.backpropagate(target)
            if self.val_frac:
                self.val_acc[epoch] = self.prediction_accuracy(self.val_data[:,:-self.num_classes], \
                                                               self.val_data[:,-self.num_classes:])
            
    def feedforward(self, inputs):
        input_size = inputs.shape[1] # Not using self.batch_size so that feedforward can take test sets as well
        self.activations[0] = inputs
        for layer in range(len(self.weights)):
            z = np.dot(self.weights[layer], self.activations[layer]) + np.tile(self.biases[layer], (1, input_size))
            self.activations[layer+1] = self.activation_function(z, self.activation_functions[layer])
    
    def activation_function(self, z, act_fun):
        if act_fun == "sigmoid":
            return 1.0/(1.0+np.exp(-z))
        if act_fun == "linear":
            return z
        if act_fun == "softmax":
            return np.exp(z)/np.sum(np.exp(z), axis=0)

    def backpropagate(self, target):
        #Calculate dcdz for the top layer
        dcdz = self.dcdz_top_layer(target) # Returns an n by m array

        #Calculate dcdw and dcdb, and then the next dcdz
        for layer in range(len(self.weights)-1, -1, -1): #Note that dcdz is above dcdw[layer] which is above a[layer]
            dcdw = np.dot(dcdz, self.activations[layer].transpose())
            self.weights[layer] = self.weights[layer] - alpha * (dcdw/self.batch_size + self.reg*self.weights[layer])
            dcdb = np.sum(dcdz, axis=1, keepdims=True)
            self.biases[layer] = self.biases[layer] - alpha * dcdb/self.batch_size
            if layer > 0: #Use current layer of dcdz to calculate next layer down of dcdz
                # Use da_lower/dz_lower * sum_over_upper(dc/dz_upper * da_lower/dz_upper)
                dcdz = self.dadz_function(layer) * np.dot(self.weights[layer].transpose(), dcdz)
    
    def dadz_function(self, layer):
        if self.activation_functions[layer-1] == "sigmoid":
            return self.activations[layer] * (1.0 - self.activations[layer])
        if self.activation_functions[layer-1] == "linear":
            return self.activations[layer]
        else: raise NameError("Not a valid activation function for layer ", layer)

    def dcdz_top_layer(self, target):
        if ((self.activation_functions[-1] == "softmax") or (self.activation_functions[-1] == "sigmoid")) and \
        (self.cost_function == "Neg Log"):
            #The math works out such that the top layer derivative dcdz = activation - target
            return self.activations[-1] - target
        elif (self.activation_functions[-1] == "sigmoid") and (self.cost_function == "Sum of Squares"):
            #For sigmoid with SS cost_fun, dcda = activation - target, dadc = a(1-a), dcdz = dcda*dadz
            dcda = self.activations[-1] - target
            return dcda * self.activations[-1] * (1 - self.activations[-1])
        elif (self.activation_functions[-1] == "linear") and (self.cost_function == "Sum of Squares"):
            return self.activations[-1] - target
        else: print('Gradient for this combination of activation function and cost function is unavailable.')

    #Function to predict a test set, output rows are data point, columns are one-hot vector predictions
    def predict(self, data):
        test_set_size = len(data)
        if self.standardize:
            data = self.standardizeFunction(data)
        self.activations = [np.zeros((layer_size, test_set_size)) for layer_size in self.sizes]
        self.feedforward(data.transpose())
        return self.activations[-1]
    
    # This only works for data with one class per data point
    def prediction_accuracy(self, data, target):
        return np.mean(np.argmax(self.predict(data), axis = 0) == np.argmax(target, axis = 1))
    
def convert_to_one_hot(vector, classes = 0):
    vector = vector.astype(np.int64)
    rows = len(vector)
    if not classes:
        classes = np.unique(vector)
    columns = len(classes)
    out = np.zeros((rows, columns))
    for i in range(rows):
        for j in range(columns):
            out[i,j] = classes[j] == vector[i]
    return out

In [12]:
dataset_name = 'MNIST'

if dataset_name == 'iris':
    from sklearn import datasets
    dataset = datasets.load_iris()
elif dataset_name == 'MNIST':
    dataset = fetch_mldata('MNIST original')
    dataset.data = dataset.data/256
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2)
y_train = convert_to_one_hot(y_train)
y_test = convert_to_one_hot(y_test)

In [13]:
hidden_layers = [30]
activation_functions = ["sigmoid", "softmax"]
cost_function = "Neg Log"
batch_size = 20
alpha = 0.1
epochs = 10
val_frac = 0.2

start = time.time()

network = FFNN(hidden_layers = hidden_layers, activation_functions = activation_functions, \
               cost_function = cost_function, data = x_train, target = y_train)
network.train(batch_size = batch_size, alpha = alpha, epochs = epochs, val_frac = val_frac, \
              standardize = False, regularization = 0)
print("accuracy is ", network.prediction_accuracy(data = x_test, target = y_test))

end = time.time()
print("Training time was ", end-start)
if val_frac:
    print("Validation set accuracy over epochs was ", network.val_acc)

epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
accuracy is  0.9545714285714286
Training time was  20.510340929031372
Validation set accuracy over epochs was  [0.90839286 0.92517857 0.933125   0.93928571 0.94455357 0.94651786
 0.94866071 0.95053571 0.95142857 0.95303571]
